In [43]:
import numpy as np
import csv
import time

In [44]:
def isEqual(theta1, theta2):
  converged = True
  for i in diff:
    # print(i)
    if abs(i) > 30:
      converged = False
  return converged

In [45]:
def get_dataset(filename):
  with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    ans = []
    for row in reader:
      values = list(map(float, row.values()))
      ans.append(values)
    return np.array(ans)

dataset = get_dataset('sample_data/titanic_data.csv')
dataset
X = dataset[:,1:]
y = dataset[:,0].reshape(887,1)
n = len(X[0])
X[:,0]= (X[:,0]- np.min(X[:,0]))/(np.max(X[:,0]) - np.min(X[:,0]))
X[:,2]= (X[:,2]- np.min(X[:,2]))/(np.max(X[:,2]) - np.min(X[:,2]))
X[:,3]= (X[:,3]- np.min(X[:,3]))/(np.max(X[:,3]) - np.min(X[:3]))
X[:,4]= (X[:,4]- np.min(X[:,4]))/(np.max(X[:,4]) - np.min(X[:,4]))
X[:,n-1]= (X[:,n-1]- np.min(X[:,n-1]))/(np.max(X[:,n-1]) - np.min(X[:,n-1]))
X

array([[1.        , 0.        , 0.27117366, 0.125     , 0.        ,
        0.01415106],
       [0.        , 1.        , 0.4722292 , 0.125     , 0.        ,
        0.13913574],
       [1.        , 1.        , 0.32143755, 0.        , 0.        ,
        0.01546857],
       ...,
       [1.        , 1.        , 0.08268409, 0.125     , 0.33333333,
        0.04577135],
       [0.        , 0.        , 0.32143755, 0.        , 0.        ,
        0.0585561 ],
       [1.        , 0.        , 0.39683338, 0.        , 0.        ,
        0.01512699]])

In [46]:
def loss(y_pred, y_label):
    loss = (y_label*np.log(y_pred) + (1-y_label)*np.log(1-y_pred)).sum()
    return loss

print(loss(np.array([0.8,0.1]).reshape(2,1), np.array([1,0]).reshape(2,1)))

-0.328504066972036


In [54]:
def gradient_ascent(y_pred,y_label,X, learning_rate, theta):    
    d = np.dot(np.transpose(X),(y_label-y_pred))
    theta += learning_rate*d
    return theta, d

# gradient_ascent()

# y_pred=pred_y(theta,X)
# print(theta)
# theta=gradient_ascent(y_pred, y, X, 1e-2, theta)
# theta


In [47]:
def sigmoid(v):
    p=1/(1+ np.exp(-v)) # p becomes 1 for large values of v .. overflow int
    p=np.minimum(p, 0.9999)
    p = np.maximum(p, 0.0001)
    return p

In [48]:
def pred_y(theta,X):
    y=np.dot(X, theta)
    # print(y)
    return sigmoid(y)
  
pred_y(theta, X).shape

(887, 1)

In [ ]:
def train(y_label, x, learning_rate, theta):
    prev_loss = 0
    while True:        
        y_pred=pred_y(theta,x)
        print("Theta = ", theta)
        cur_loss = loss(y_pred, y_label)
        print("Loss = ", cur_loss)
        theta, derivative = gradient_ascent(y_pred, y_label, x, learning_rate, theta)         
        print("derivative = ", derivative)
        if abs(derivative[0]) < 0.01 and abs(derivative[1]) < 0.01 and abs(derivative[2]) < 0.01 and abs(derivative[3]) < 0.01 and abs(derivative[4]) < 0.01 and abs(derivative[5]) < 0.01 :
          break        
        prev_loss = cur_loss        
    return theta

theta = np.array([1.0,3.3,2.1,6.3,3.4,4.1]).reshape(6,1)        
start = time.time()
train(y, X,  0.001, theta)
end = time.time()
print("Training time -> ", end - start, "seconds") 


In [51]:
theta

array([[-1.57840018],
       [ 2.78111798],
       [-1.69483831],
       [-2.56940522],
       [-0.69231501],
       [ 3.15283636]])

In [40]:
def fisher(theta, x):
  return np.exp(-theta.T.dot(x))/((1+np.exp(-theta.T.dot(x)))**2)*x.dot(x.T)

o = np.zeros(shape=(6,6))

for i in range(len(X)):
  x = X[i].reshape(6,-1)
  o += fisher(theta, x)

fisher_inv = np.linalg.inv(o)
fisher_inv

array([[ 0.03508576, -0.01306604, -0.0353113 , -0.03573973, -0.01777283,
         0.08153019],
       [-0.01306604,  0.0377613 , -0.00932018, -0.02515883, -0.02740611,
        -0.00606216],
       [-0.0353113 , -0.00932018,  0.12519292,  0.05112621,  0.02571607,
        -0.26109075],
       [-0.03573973, -0.02515883,  0.05112621,  0.68232714, -0.13077865,
        -0.28077633],
       [-0.01777283, -0.02740611,  0.02571607, -0.13077865,  0.47176689,
        -0.2209946 ],
       [ 0.08153019, -0.00606216, -0.26109075, -0.28077633, -0.2209946 ,
         1.59442956]])

In [53]:
my_x = np.array([0.0, 0, 0.5601910027645137, 0.25, 0.3333333333333333, 0.03903740017160841]).reshape(1,6) # should we normalize this as well?
# my_x = np.array([0.0, 1, 0.5601910027645137, 0.25, 0.3333333333333333, 0.03903740017160841]).reshape(1,6) # should we normalize this as well?
pr = pred_y(theta, my_x)
pr

array([[0.1545335]])

In [41]:
my_x=my_x.reshape(6,1)
(my_x.T.dot(fisher_inv)).dot(my_x)

array([[0.11625774]])

In [ ]:
def train(y_label,x, learning_rate, theta, iterations):
    list_cost=[]
    i = 0

    while i<iterations:
        i+=1
        y_pred=pred_y(theta,x)
        print(theta)
        print("loss= ", loss(y_pred, y_label))
        # print(theta)
        theta_new=gradient_ascent(y_pred, y_label, x, learning_rate, theta)
        
        # diff = 
        # print(np.isclose(theta,theta_new,rtol=1,atol=0))
        # if isEqual(theta, theta_new):
        #   break
        theta = theta_new
        # if i%100==0:
            # print("n iteration",i)
            # print("y_label:",y_pred)
            # print("theta:",theta)
        
        # break
        # list_cost.append(cost)
    # print("final cost list: ",list_cost)
    # print(i)
    return theta

# theta = np.random.normal(0, 1, size=6).reshape(6,1)
theta = np.array([1.0,3.3,2.1,6.3,3.4,4.1]).reshape(6,1)
# theta = np.array([0.08910022,  3.46893814, -7.29130247,  5.88985005,  3.22153113,  0.59336014]).reshape(6,1)
# theta = np.array([ 0.44488621,  3.62349863, -0.23738928,  5.69266761,  3.15389619,
#         -0.01068158]).reshape(6,1)
# theta = np.array([-0.73437146,  1.08034312, -0.57048562, -0.36618415,  0.03922978,
        #  0.59767026]).reshape(6,1)
# theta = np.array([-0.73437146,  1.08034312, -0.57048562, -0.36618415,  0.03922978,
#          0.59767026]).reshape(6,1)
        
train(y, X,  0.01, theta, 10000)

In [42]:
my_x.T.dot(theta)

array([[-1.69947761]])

In [ ]:
for j,i in enumerate(pr):
  if i >= 0.5:
    pr[j] = 1.0
  else:
    pr[j] = 0.0
pr

In [ ]:
c = 0
for i in range(len(pr)):
  # print
  if pr[i] == y[i]:
    c +=1
print(c)
p # 72% accuracy 

643


In [ ]:
n = len(X[0])

In [ ]:
theta = np.random.normal(0, 1, size=n)
theta = theta.reshape(6,1)

array([ 0.58016022,  0.27655519,  0.76057244, -0.89901507, -1.28645374,
        0.3946704 ])

In [ ]:
a = np.array([1,2])
b = np.array([1.1,3.7])
diff = a -b

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
logreg = LogisticRegression()
logreg.fit(X,y.ravel())


LogisticRegression()

In [ ]:
logreg.predict(X)

In [ ]:
logreg.

<bound method ClassifierMixin.score of LogisticRegression()>

In [ ]:
x = np.array([[-0.73437146],
 [ 1.08034312],
 [-0.57048562],
 [-0.36618415],
 [ 0.03922978],
 [ 0.59767026]])
x.T

array([[-0.73437146,  1.08034312, -0.57048562, -0.36618415,  0.03922978,
         0.59767026]])

In [ ]:
np.exp(theta.v)

array([[ 0.29680176,  3.61731153, -0.13353561,  5.53865928,  3.06593566,
         0.00903959]])

In [34]:
dataset = get_dataset('sample_data/titanic_data.csv')
dataset
X = dataset[:,1:]

l = [1,0,45,2,2,20]
l[0] = (l[0] - np.min(X[:,0]))/(np.max(X[:,0]) - np.min(X[:,0]))
l[2] = (l[2] - np.min(X[:,2]))/(np.max(X[:,2]) - np.min(X[:,2]))
l[3] = (l[3] - np.min(X[:,3]))/(np.max(X[:,3]) - np.min(X[:,3]))
l[4] = (l[4] - np.min(X[:,4]))/(np.max(X[:,4]) - np.min(X[:,4]))
l[n-1] = (l[n-1] - np.min(X[:,n-1]))/(np.max(X[:,n-1]) - np.min(X[:,n-1]))

l
#

[0.0, 0, 0.5601910027645137, 0.25, 0.3333333333333333, 0.03903740017160841]

In [35]:
l = [0.0, 0, 0.5601910027645137, 0.25, 0.3333333333333333, 0.03903740017160841]
